# Data analysis is all you need firstly!

## load package

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt
import os
import string
from collections import defaultdict
from transformers import AutoTokenizer
from tqdm import tqdm


In [ ]:
tokenizer=AutoTokenizer.from_pretrained('../input/bert-base-uncased')

In [ ]:
data_text_path ='../input/feedback-prize-2021/train'
data_label_path ='../input/feedback-prize-2021/train.csv'

In [ ]:
df_train_label = pd.read_csv('../input/feedback-prize-2021/train.csv')

In [ ]:
df_train_label.head(2)

In [ ]:
def get_tokenized_len(text):
    return len(tokenizer.encode(text))

## ans type

In [ ]:
df_train_label['discourse_type'].value_counts()

In [ ]:
df_train_label['discourse_start'].hist(bins=[0,200,500,1000,2000,3000,4000,5000,6000])

In [ ]:
df_train_label['discourse_end'].hist(bins=[0,200,500,1000,2000,3000,4000,5000,6000])

## ans char len

In [ ]:
(df_train_label['discourse_end']-df_train_label['discourse_start']).hist(bins=[0,100,200,500,1000,2000])

## ans word len

In [ ]:
def get_split_len(textstr):
    return len(textstr.strip().split())

df_train_label['predictionstring'].apply(get_split_len).hist(bins=[0,50,100,150,200,500])

## per discoures_text len after tokenized

In [ ]:
df_train_label['discoures_text_len'] = df_train_label['discourse_text'].apply(get_tokenized_len)

In [ ]:
df_train_label.hist(column=['discoures_text_len'])

## processing

In [ ]:
df_group = df_train_label.groupby('id')

In [ ]:
def f(x):
    #print(len(x['id']))
    return pd.Series({    
                          'discourse_id':x['discourse_id'].values.reshape(-1).tolist(),
                         'discourse_start':x['discourse_start'].values.reshape(-1).tolist(),
                         'discourse_end':x['discourse_end'].values.reshape(-1).tolist(),
                          'discourse_text':x['discourse_text'].values.reshape(-1).tolist(),
                          'discourse_type':x['discourse_type'].values.reshape(-1).tolist(),
                         "predictionstring":x['predictionstring'].values.reshape(-1).tolist()
            
                         }).T


In [ ]:
df_group_recol = df_group.apply(f)

In [ ]:
#df_group_recol.index 

In [ ]:
df_group_recol.reset_index(level=0, inplace=True)

In [ ]:
df_group_recol.head()

In [ ]:
len(df_group_recol)

In [ ]:
def get_text_from_afile(text_path):
    with open(os.path.join("../input/feedback-prize-2021/train",text_path+'.txt'), 'r') as f:
        return (f.read())

In [ ]:
df_group_recol['text'] = df_group_recol['id'].apply(get_text_from_afile)
    
    

In [ ]:
df_group_recol.head()

## ans char position check

In [ ]:
inequal_count =0 
for index, irow in df_group_recol.iterrows():
    start_list = irow['discourse_start']
    end_list =irow['discourse_end']
    discourse_text_list = irow['discourse_text']

    split_text=irow['text'].split()

    for istart, iend,ians in zip(start_list, end_list,discourse_text_list):
        

        extracted_str = irow['text'][int(istart):int(iend)].strip().strip(string.punctuation)#.strip().strip(string.punctuation).strip()
        ians_p = ians.strip().strip(string.punctuation)#.strip().strip(string.punctuation).strip()
        if extracted_str!= ians_p:
            inequal_count+=1
            #print('*'*20)
            #print(irow['text'][int(istart):int(iend)], ians,sep='**')


In [ ]:
inequal_count

## ans predictionstring check

In [ ]:
inequal_count =0 
for index, irow in df_group_recol.iterrows():
    start_list = irow['discourse_start']
    end_list =irow['discourse_end']
    discourse_text_list = irow['discourse_text']
    predictionstring_list = irow['predictionstring']
    split_text=irow['text'].split()
    
    for ians,predictionstring in zip(discourse_text_list,predictionstring_list):
        predictionstring_split =predictionstring.strip().split()
        word_start_index = int(predictionstring_split[0])
        word_end_idnex = int(predictionstring_split[-1])
        ans_from_wordidnex = ' '.join(split_text[word_start_index:word_end_idnex+1]).strip().strip(string.punctuation).strip()
        ians_p = " ".join(ians.split()).strip().strip(string.punctuation).strip()
        if ans_from_wordidnex!=ians_p:
            inequal_count+=1
            #print('*'*20)
            #print(ans_from_wordidnex,ians_p,'-pair_end-\n',sep='*******',)

In [ ]:
inequal_count

## ans split check

In [ ]:
inequal_count =0
for index, irow in df_group_recol.iterrows():
    start_list = irow['discourse_start']
    end_list =irow['discourse_end']
    discourse_text_list = irow['discourse_text']
    predictionstring_list = irow['predictionstring']
    
    split_text=irow['text'].split()

    wordindex2charindex={}
    ipos=0
    new_format_text = ""
    for i_index,iword in enumerate(split_text):
        wordindex2charindex[i_index]=(ipos,ipos+len(iword)-1)
        # space
        ipos= ipos+len(iword)+1
        new_format_text+=(iword+' ')
    for ians,predictionstring in zip(discourse_text_list,predictionstring_list):
        predictionstring_split =predictionstring.strip().split()
        word_start_index = int(predictionstring_split[0])
        word_end_idnex = int(predictionstring_split[-1])
        extracted_str=new_format_text[wordindex2charindex[word_start_index][0]:wordindex2charindex[word_end_idnex][1]+1]
        if extracted_str.strip().strip(string.punctuation)!= " ".join(ians.split()).strip().strip(string.punctuation):
                inequal_count+=1


In [ ]:
inequal_count

## label count per doc

In [ ]:
df_group_recol.head(2)

In [ ]:

df_group_recol['label_count'] =df_group_recol['discourse_type'].apply(len)

In [ ]:
df_group_recol.hist(column=['label_count'])

## each label type count per doc

In [ ]:
label_type = list(df_train_label['discourse_type'].value_counts().keys())

In [ ]:
def perlabel_count(label_list):
    dd_dict = defaultdict(lambda:0)
    for ilabel in label_list:
        dd_dict[ilabel]+=1
    return [ dd_dict[il_type] for il_type in label_type]

In [ ]:
df_group_recol['perlabel_count'] = df_group_recol['discourse_type'].apply(perlabel_count)

In [ ]:
df_group_recol.head(1)

In [ ]:
def display_multiple_img(df_list, rows, cols,title):
    '''
    ref: https://stackoverflow.com/questions/38082602/plotting-multiple-different-plots-in-one-figure-using-seaborn
    
    '''
    figure, all_ax = plt.subplots(nrows=rows,ncols=cols,figsize=(16,3))
    plt.suptitle(title, fontsize=20)
    for i in range(rows*cols):
        df_list[i].hist(ax=all_ax.ravel()[i])
    plt.tight_layout()
    plt.show()

In [ ]:
type_count_array=np.array(df_group_recol['perlabel_count'].values.reshape(-1).tolist())

In [ ]:
df_list = [pd.DataFrame({itype:type_count_array[:,i_index]}) for i_index, itype in enumerate(label_type)]

In [ ]:
display_multiple_img(df_list, 1, 7,label_type)

## len after tokenied

In [ ]:
df_group_recol['tokenizer_len'] = df_group_recol['text'].apply(get_tokenized_len)

In [ ]:
df_group_recol.hist(column=['tokenizer_len'])

## unkown word

In [ ]:
tokenizer.unk_token_id

In [ ]:


unktoken_dict={}
for text in tqdm(df_group_recol['text']):
    tokenized_examples =tokenizer(
            text,  # QA第一个句子的列表
            max_length=512,  # 最大长度
            stride=0,  # 是不同窗口 重叠的长度，不是滑动的长度
            return_overflowing_tokens=True,  # 返回每个span 样例对应的 question-context的样本序号 如[0,0,0,1,1,1]
            return_offsets_mapping=True,  # 返回token对应的char的位置
            padding="max_length",  # 设置padding，“max_length”表示设置为 当前的max_length的值，True表示paddingg到模型的最长长度，False不padding
        )
    for index in range(len(tokenized_examples['input_ids'][0])):
        if tokenized_examples['input_ids'][0][index] ==tokenizer.unk_token_id:
            #print(tokenized_examples['offset_mapping'][0][index])
            
            span  = str(text[tokenized_examples['offset_mapping'][0][index][0]:tokenized_examples['offset_mapping'][0][index][1]])
            if span not in unktoken_dict:
                unktoken_dict[span] =0 
            try:
                unktoken_dict[span]+=1
            except:
                print(span)
                pass
        #print(tokenized_examples['input_ids'])
        #print(tokenized_examples['offset_mapping'])
        #print(tokenized_examples.sequence_ids(0))
    
print(len((unktoken_dict)))
print(unktoken_dict)